# 本セクションの目次
1. 分散処理とは
2. PySparkとは
3. ノートブックとは
4. Spark(PySpark)がデータ操作で利用するもの
5. JSONのデータ読み込み
6. CSVのデータ読み込み
7. データフレームを操作する
8. カラムナーフォーマット
9. 圧縮形式とファイルフォーマット
10. 行指向フォーマット
11. パーティションとダイナミックパーティション
12. スモールファイルとデータスキュー

# 分散処理とは？

分散処理とは、複数の端末にまたがって処理を行うことです。  
スレッド処理とは異なり複数台にまたがっていることがポイントです。

例えば、「This is good」「That are very good」のgoodの数を数えるプログラムを考えるときの場合を考えてみます。

## スレッド処理の場合
一台のパソコン（端末）の中でスレッドを立ち上げ処理を行います。

端末1  
　 - スレッドの呼び出しもと  
    - スレッド1 「This is good」のgoodの数を数える  
    - スレッド2 「That are very good」のgoodの数を数える  
-> スレッド１、２がそれぞれgoodの数を数えてスレッドの呼び出し元にてそれぞれのスレッドの合計を取得し結果を表示する

## 分散処理の場合
複数台(今回は3台)で処理を行います。

コントローラーノード(今回だとドライバー)   
　- 端末1,端末2に指示を出して処理をさせる  
　- 端末1,端末2からの処理結果を受け取りユーザに返却する  

端末1
  - 「This is good」のgoodの数を数える

端末2
  - 「That are very good」のgoodの数を数える

分散処理の特徴は、スレッド処理としてノードを(実質)無限にスケールすることが可能な点です。  
仮にCPUやメモリが足りなければ端末3,4,,,,と増やしていくことで処理のボトルネックを解決することが可能になるという理論です。  
一方で、スレッド処理は一つの端末内で実行されるため、無限にスケールすることができません(CPU/メモリに限界がある)  

今回のコースは、データも小さく端末が一台(コントローラーノードと端末1が同居している状態)ですが実際の環境で構築する(された)環境を利用する際は複数台であることが一般的です。

# PySparkとは？
PySparkとは分散処理を実現するエンジン(フレームワークの様なもの)です。  
ビッグデータの世界では、一つのノードで処理を行うということはほとんどなく複数台のノードで一つの仕事を行い結果を出すということがしばしば行われます。  

その処理を比較的簡単に行なってくれるのがSparkであり、Pythonとの組み合わせて使うことでPySparkと呼ばれています。  
他にはScalaとの組み合わせやJavaとの組み合わせが可能ですが、ビッグデータ分析を行うという観点からPythonが多く利用されています。

単純に複数台並べて多くのリクエストを捌く様なWebシステムというわけではなくて、一つの大きな仕事をを処理するためのフレームワークと考えると良いと思います。

## ちょっとだけSpark Internal
ちょっとだけInternalということで、Sparkで利用される言葉について整理をしていきましょう。

Sparkは大きく分けると
- ドライバー
- エグゼキューター

に分かれています。

ドライバーはユーザからの処理(SQLの実行など)を受け付けエグゼキューターに引き渡します。  
そして、エグゼキュータはその処理を実行するという流れです。  

ドライバーはAノードエグゼキューターはA,B、Cノードといった様に分散処理を実現していきます。

# ノートブックとは？

ノートブックとはGUIで視覚的に分析を可能にした、まさにノートの様な分析環境です。  
プログラムやこの様にマークダウンを記載する場所を「セル」と呼びます。  
セルにプログラムやその説明を記載しながら状況を残していきます。

分析の結果をノートの様に一個ずつ残していけるからノートブックと呼ばれています。

このノートブック自体は他の人に共有することも可能で、実行したグラフの結果などをそのまま表示させてGitにコミットを行うことによって  
共有を行います。

今回はVSCodeのノートブック環境を利用していますが他にも

- jupyter notebook
- EMR notebook
- zeppeline

などなど、クラウド環境で提供されているものもあります。

# Spark(PySpark)がデータ操作で利用するもの

Sparkがデータを操作するときに利用するものは2つ(正確には3つ)あります。  

- SQL
- DataFrame
- RDD

今回のコースは、SQLとDataFrameをそれぞれ対比させながら紹介をおこなっていこうと思います。  
また、あまり出番はないのですが、RDDについても少し触れてみようと思います。

SQLとはデータを読み込みし、そのデータに対して仮想的なテーブルを作成しSQLを使ってデータ操作を行います。  
DataFrameとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います。  
RDDとはデータを読み込みし、そのデータに対してプログラムチックにデータ操作を行います(DataFrameよりより、低レベルのAPIを提供します)。

Sparkには2つの読み込みタイプが存在しています  
- スキーマオンリード(事前のテーブル定義がなくてもデータを読み込んで処理が可能)  
- スキーマオンライト(データを読み込むためには、事前のテーブル定義が必要)  

既存のテーブルからデータを読み取ることも可能ですしなくても問題ありません。

詳しくはセクション3でみていきましょう。

# どのセクションでも登場する前準備

どのセクションでも、データ処理を開始する前のおまじないがあります。

In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,org.apache.spark:spark-avro_2.12:3.2.1") \
    .enableHiveSupport() \
    .getOrCreate()

# Spark SessionはJavaでいうインスタンスの様なもので、spark(=分散処理しますよ)というマークだと考えればOK
# それ以外はconfigで細かな設定が可能
# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6220e1ed-e990-4ea8-9665-e724ff0c573f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# Jsonのデータ読み込み
SparkでJsonのデータを読み込んでみましょう。

In [3]:
# Jsonデータの読み込み
json_df=spark.read.json("./dataset/jinko.json")
json_df.show(n=10)

+----+----+-----+------------+----------+------+-------+--------+------+
| chu|code|gengo|jinko_female|jinko_male|kenmei|seireki|   sokei|wareki|
+----+----+-----+------------+----------+------+-------+--------+------+
|null|  00| 大正|    27918868|  28044185|  全国|   1920|55963053|     9|
|null|  01| 大正|     1114861|   1244322|北海道|   1920| 2359183|     9|
|null|  02| 大正|      375161|    381293|青森県|   1920|  756454|     9|
|null|  03| 大正|      424471|    421069|岩手県|   1920|  845540|     9|
|null|  04| 大正|      476459|    485309|宮城県|   1920|  961768|     9|
|null|  05| 大正|      444855|    453682|秋田県|   1920|  898537|     9|
|null|  06| 大正|      490597|    478328|山形県|   1920|  968925|     9|
|null|  07| 大正|      689225|    673525|福島県|   1920| 1362750|     9|
|null|  08| 大正|      688272|    662128|茨城県|   1920| 1350400|     9|
|null|  09| 大正|      532224|    514255|栃木県|   1920| 1046479|     9|
+----+----+-----+------------+----------+------+-------+--------+------+
only showing top 10 rows



# CSVのデータ読み込み
CSV(TSV)の読み込みは非常にパターンが多いです。
ひとつを例にとってパターンを見てみましょう。

多すぎるので全て、紹介できませんがいくつかオプションをみていきましょう。

https://spark.apache.org/docs/latest/sql-data-sources-csv.html

- inferSchema(型をデータの中から推論するかどうか？推論なのでデータの中身によって毎度変わる可能性があるので注意)
- lineSep(sep),(csvであれば','でtsvであればタブ)
- header(一行目をカラム行としてみなすかどうか？)
- multiline(カラムで改行されたりした場合に、それを一行としてみるかどうか)
- Schema(スーキーマを設定する)
- encoding(読み込むエンコードを設定する)

In [4]:
#データソースの読み込み
#sep='\t'とすればtsvでも読み込みが可能です
#multiLineは、CSVやTSVの各カラムに改行が含まれていた時の対策です。
df=spark.read.option("multiLine", "true").option("encoding", "SJIS").csv("./dataset/jinko.csv", header=True, sep=',', inferSchema=False)
df.show()

+--------------+----------+----+----------+----------+----+------------+----------+----------+
|都道府県コード|都道府県名|元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|人口（女）|
+--------------+----------+----+----------+----------+----+------------+----------+----------+
|            00|      全国|大正|         9|      1920|null|    55963053|  28044185|  27918868|
|            01|    北海道|大正|         9|      1920|null|     2359183|   1244322|   1114861|
|            02|    青森県|大正|         9|      1920|null|      756454|    381293|    375161|
|            03|    岩手県|大正|         9|      1920|null|      845540|    421069|    424471|
|            04|    宮城県|大正|         9|      1920|null|      961768|    485309|    476459|
|            05|    秋田県|大正|         9|      1920|null|      898537|    453682|    444855|
|            06|    山形県|大正|         9|      1920|null|      968925|    478328|    490597|
|            07|    福島県|大正|         9|      1920|null|     1362750|    673525|    689225|
|            08|    茨城県|大正|        

In [5]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

#スキーマ設定をしていきましょう
struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])

df_csv=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("./dataset/jinko.csv", header=False, sep=',', inferSchema=False, schema=struct)

df_csv.show(truncate=False,n=4)

df_csv=df_csv.filter(df_csv['code'] != '都道府県コード')
df_csv.show(truncate=False,n=4)

# headerやsepなどはoptionで渡すことも可能ですが、csv関数の引数として渡すことも可能です。

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|code          |kenmei    |gengo|wareki    |seireki   |chu |sokei       |jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名|元号 |和暦（年）|西暦（年）|注  |人口（総数）|人口（男）|人口（女）  |
|00            |全国      |大正 |9         |1920      |null|55963053    |28044185  |27918868    |
|01            |北海道    |大正 |9         |1920      |null|2359183     |1244322   |1114861     |
|02            |青森県    |大正 |9         |1920      |null|756454      |381293    |375161      |
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
only showing top 4 rows

+----+------+-----+------+-------+----+--------+----------+------------+
|code|kenmei|gengo|wareki|seireki|chu |sokei   |jinko_male|jinko_female|
+----+------+-----+------+-------+----+--------+----------+------------+
|00  |全国  |大正 |9     |1920 

# データフレーム
df_csvやjson_dfがデータフレームと呼ばれる変数

データを枠組み(フレーム)として扱うことができるのがデータフレームです。  
データフレームの作成方法は

- SQLから生成する
- データを読み込んで生成する
- プログラム的に作成する

SQLから生成はSQLで取得した結果がデータフレームで返却されるます  

データを読み込んで生成する場合はCSVやJSon、Parquet、Avroなどのファイルを読み込んでデータフレームに取り込みを行うことが可能  

プログラム的に作成するのは、空のデータフレームが欲しい時など

## データフレームの操作
データフレームに対する操作はたくさんあるのですが、今回はよく使う

- withColumnsとalias
- When(otherwise)
- null関係の操作(fillna)
- 条件絞り込み(filter)
- Subtract

について紹介していきたいと思います。

集計関数や結合については別のセクションで紹介します。

In [11]:
#ちなみにdf_csv.と打つと候補がたくさん出てきます

# WithCloumnsはカラムを付与する作業のことです
# CSVのデータフレームに対してカラムを付与してみましょう
from pyspark.sql.functions import lit,when
df_csv.withColumn("hoge",lit("1")).show(n=2)

# when
# IF文みたいなものです
df_csv.withColumn("peke",when(df_csv.gengo =='大正',lit("2"))).show(n=2)

#下の方みるとわかるのですが平成がNUllです
df_csv.withColumn("peke",when(df_csv.gengo =='大正',lit("2")) \
.when(df_csv.gengo == '昭和',df_csv.wareki)).filter(df_csv.kenmei == '全国').show(n=20)

df_csv.select(df_csv.code.alias('peke')).show(n=4)

+----+------+-----+------+-------+----+--------+----------+------------+----+
|code|kenmei|gengo|wareki|seireki| chu|   sokei|jinko_male|jinko_female|hoge|
+----+------+-----+------+-------+----+--------+----------+------------+----+
|  00|  全国| 大正|     9|   1920|null|55963053|  28044185|    27918868|   1|
|  01|北海道| 大正|     9|   1920|null| 2359183|   1244322|     1114861|   1|
+----+------+-----+------+-------+----+--------+----------+------------+----+
only showing top 2 rows

+----+------+-----+------+-------+----+--------+----------+------------+----+
|code|kenmei|gengo|wareki|seireki| chu|   sokei|jinko_male|jinko_female|peke|
+----+------+-----+------+-------+----+--------+----------+------------+----+
|  00|  全国| 大正|     9|   1920|null|55963053|  28044185|    27918868|   2|
|  01|北海道| 大正|     9|   1920|null| 2359183|   1244322|     1114861|   2|
+----+------+-----+------+-------+----+--------+----------+------------+----+
only showing top 2 rows

+----+------+-----+------+------

In [46]:
#　平成がNullではなくないりました
df_csv.withColumn("peke",when(df_csv.gengo =='大正',lit("2")) \
.when(df_csv.gengo == '昭和',df_csv.wareki).otherwise("99999")).filter(df_csv.kenmei == '全国').show(n=20)

+----+------+-----+------+-------+----+---------+----------+------------+-----+
|code|kenmei|gengo|wareki|seireki| chu|    sokei|jinko_male|jinko_female| peke|
+----+------+-----+------+-------+----+---------+----------+------------+-----+
|  00|  全国| 大正|     9|   1920|null| 55963053|  28044185|    27918868|    2|
|  00|  全国| 大正|    14|   1925|null| 59736822|  30013109|    29723713|    2|
|  00|  全国| 昭和|     5|   1930|null| 64450005|  32390155|    32059850|    5|
|  00|  全国| 昭和|    10|   1935|null| 69254148|  34734133|    34520015|   10|
|  00|  全国| 昭和|    15|   1940|null| 73114308|  36566010|    36548298|   15|
|  00|  全国| 昭和|    20|   1945|  1)| 71998104|  33894059|    38104045|   20|
|  00|  全国| 昭和|    25|   1950|null| 84114574|  41241192|    42873382|   25|
|  00|  全国| 昭和|    30|   1955|null| 90076594|  44242657|    45833937|   30|
|  00|  全国| 昭和|    35|   1960|  2)| 94301623|  46300445|    48001178|   35|
|  00|  全国| 昭和|    40|   1965|null| 99209137|  48692138|    50516999|   40|


In [53]:
# nullを処理する方法は他にもあります
# fillna
df_csv.withColumn("peke",when(df_csv.gengo =='大正',lit("2")) \
.when(df_csv.gengo == '昭和',df_csv.wareki)).fillna({"peke":"a","chu":"1"}).filter(df_csv.kenmei == '全国').show(n=20)

+----+------+-----+------+-------+---+---------+----------+------------+----+
|code|kenmei|gengo|wareki|seireki|chu|    sokei|jinko_male|jinko_female|peke|
+----+------+-----+------+-------+---+---------+----------+------------+----+
|  00|  全国| 大正|     9|   1920|  1| 55963053|  28044185|    27918868|   2|
|  00|  全国| 大正|    14|   1925|  1| 59736822|  30013109|    29723713|   2|
|  00|  全国| 昭和|     5|   1930|  1| 64450005|  32390155|    32059850|   5|
|  00|  全国| 昭和|    10|   1935|  1| 69254148|  34734133|    34520015|  10|
|  00|  全国| 昭和|    15|   1940|  1| 73114308|  36566010|    36548298|  15|
|  00|  全国| 昭和|    20|   1945| 1)| 71998104|  33894059|    38104045|  20|
|  00|  全国| 昭和|    25|   1950|  1| 84114574|  41241192|    42873382|  25|
|  00|  全国| 昭和|    30|   1955|  1| 90076594|  44242657|    45833937|  30|
|  00|  全国| 昭和|    35|   1960| 2)| 94301623|  46300445|    48001178|  35|
|  00|  全国| 昭和|    40|   1965|  1| 99209137|  48692138|    50516999|  40|
|  00|  全国| 昭和|    45|   1

In [8]:
# 最後はSubtract
# データの一致などを計算する際に利用されることが多いです。
# 例えばデータ移行などで、移行前と移行後でデータを比較したい時など

df_csv.subtract(json_df).show()

+----+--------+-----+------+-------+----+-------+----------+------------+
|code|  kenmei|gengo|wareki|seireki| chu|  sokei|jinko_male|jinko_female|
+----+--------+-----+------+-------+----+-------+----------+------------+
|  30|和歌山県| 昭和|    55|   1980|null|1087012|    523467|      563545|
|  05|  秋田県| 平成|     7|   1995|null|1213667|    577535|      636132|
|  35|  山口県| 平成|    17|   2005|null|1492606|    703721|      788885|
|  25|  滋賀県| 昭和|     5|   1930|null| 691631|    337016|      354615|
|  26|  京都府| 昭和|     5|   1930|null|1552832|    792420|      760412|
|  47|  沖縄県| 昭和|    25|   1950|null| 914937|    429432|      485505|
|  16|  富山県| 平成|     7|   1995|null|1123125|    540921|      582204|
|  02|  青森県| 平成|    12|   2000|null|1475728|    702573|      773155|
|  23|  愛知県| 平成|    27|   2015|null|7483128|   3740844|     3742284|
|  30|和歌山県| 大正|    14|   1925|null| 787511|    392191|      395320|
|  20|  長野県| 昭和|     5|   1930|null|1717118|    832312|      884806|
|  27|  大阪府| 昭和|     

# カラムナーフォーマット
ここまでで読み込んだJsonやCsvはローデータとよばれる、データ基盤では扱いづらい部類に入るファイルです。  
扱いづらい理由はファイルがスプリッタブルかどうか？という点が関わっています。  

スプリッタブルであればデータを複数端末で分割して処理することが可能ですし、そうでなければ分割して処理をすることができないため非効率が発生します。  

カラムナーフォーマットと次のレクチャーで紹介するAvro(行指向フォーマット)はいずれもスプリッタブルなファイルです。  
そのため、CSVやJSON形式のままデータ基盤で活用を続けるのは悪手なので、必ずカラムナーフォーマット、行指向フォーマットへ変換する処理を挟みましょう。  

カラムナーフォーマットとは、分析に特化したファイル形式です。  
列ごとにデータをまとめて保存することによって、データを圧縮し検索速度(特にGroup byなどの集計構文)をあげています。  
また、カラムナーフォーマットは内部にカラム情報を保持しているためセクション2で紹介したスキーマオンリード機能を使ってスキーマ定義を読み出すことができます。

# 圧縮形式とファイルフォーマット

前のレクチャーではスプリッタブルかどうかが重要であるとお伝えしました。  
ここで一度、ファイルと圧縮形式の組み合わせについてみていきましょう。

```
        paruqet avro csv/json
gz       Y      Y      N
snappy   Y      Y      -
bz2      -      -      Y
圧縮なし  Y      Y      N

```

大事なのは、csvやjsonの場合はbz2を利用しましょう。

# 行指向フォーマット
先ほどはカラムナーフォーマットを紹介しましたが、次は行指向フォーマットを紹介について紹介していきます。  

よく利用されているRDBも行指向の仕組みを持ったデータシステムです。  
RDBはレコードの追加が得意です。  
 
そして、ビッグデータはレコードを一件つづ追加していく様な処理が苦手でしたが行指向フォーマットのAvroの登場によってレコードの追加が頻繁に発生する処理  
つまりストリーミング処理にも適用することが可能になりました。  

まず覚えてほしいのは、Avroはメインとしてストリーミング処理の利用に向いているということです（バッチ処理にも使うことが可能です）  


ビッグデータにおけるストリーミングについて詳しく知りたい方は  
「データサイエンスのためのストリーミング前処理入門　PythonとSparkで始めるビッグデータストリーミング処理入門」
を是非受講ください。

# パーティションとダイナミックパーティション

データはパーティションと呼ばれれる区切りにデータを保存していくことが可能です。

```
テーブル
    パーティション1
        パーティション1に関するデータ

    パーティション2
        パーティション2に関するデータ
```

今回は、パーティションとしてkenmeiを利用して保存をしていこうと思います。
また、ファイルの出力としてParquet/Avroでそれぞれ出力を行ってみたいと思います。

## ダイナミックパーティション
ダイナミックパーティションとはデータをもとにパーティションを作成することを指します。  
Hiveなどで使われる言葉ですが、データをもとに振り分けるんだな。  
とだけ理解しておけばいいと思います。

In [12]:
# repartitionはパーティション配下のファイルの数をまとめるものです
# 次のスモールデータとデータスキューネスに関係していきます。


# Parquet
# パーティションなしの場合
df_csv.repartition(2).write.mode('overwrite').parquet("/home/pyspark/super_crush_course.db/parquet_table_with_no_partition")


# Avroでファイルを出力してみよう
# パーティションなしの場合
df_csv.repartition(2).write.mode('overwrite').format("avro").save("/home/pyspark/super_crush_course.db/avro_table_with_no_partition")


In [13]:
# パーティションありの場合
df_csv.repartition(2).write.partitionBy('kenmei').mode('overwrite').parquet("/home/pyspark/super_crush_course.db/parquet_table_with_partition")

# パーティションありの場合
df_csv.repartition(2).write.partitionBy('kenmei').mode('overwrite').format("avro").save("/home/pyspark/super_crush_course.db/avro_table_with_partition")


In [15]:
!ls -al /home/pyspark/super_crush_course.db/parquet_table_with_no_partition
!echo '=============='
!ls -al /home/pyspark/super_crush_course.db/parquet_table_with_partition/kenmei=三重県

total 52
drwxr-xr-x 2 pyspark pyspark  4096 Jan  7 02:10 .
drwxr-xr-x 6 pyspark pyspark  4096 Jan  7 02:10 ..
-rw-r--r-- 1 pyspark pyspark     8 Jan  7 02:10 ._SUCCESS.crc
-rw-r--r-- 1 pyspark pyspark   136 Jan  7 02:10 .part-00000-d7641e03-9b73-452a-9dab-f09056a6698a-c000.snappy.parquet.crc
-rw-r--r-- 1 pyspark pyspark   136 Jan  7 02:10 .part-00001-d7641e03-9b73-452a-9dab-f09056a6698a-c000.snappy.parquet.crc
-rw-r--r-- 1 pyspark pyspark     0 Jan  7 02:10 _SUCCESS
-rw-r--r-- 1 pyspark pyspark 16165 Jan  7 02:10 part-00000-d7641e03-9b73-452a-9dab-f09056a6698a-c000.snappy.parquet
-rw-r--r-- 1 pyspark pyspark 16288 Jan  7 02:10 part-00001-d7641e03-9b73-452a-9dab-f09056a6698a-c000.snappy.parquet
total 216
drwxr-xr-x 53 pyspark pyspark 4096 Jan  7 02:10  .
drwxr-xr-x  6 pyspark pyspark 4096 Jan  7 02:10  ..
-rw-r--r--  1 pyspark pyspark    8 Jan  7 02:10  ._SUCCESS.crc
-rw-r--r--  1 pyspark pyspark    0 Jan  7 02:10  _SUCCESS
drwxr-xr-x  2 pyspark pyspark 4096 Jan  7 02:10 'kenmei=__HIVE_

# スモールデータとデータスキューネス
ここで一点気をつけなければならないのが、スモールファイルとデータスキューネスです。

ビッグデータの世界では、データが小さすぎると処理のボトルネックになります。

前のレクチャーのパーティションに分けることは確かに有効な手段なのですが、あまりに小さくパーティションを作成しすぎるとファイルのサイズが小さくなってしまうため  
パーティションお分け方には要注意です。

また、データの偏り(スキュー)にも気をつける必要があります。  
Aのパーティションだけが異常にデータ量が大きくBのパーティションが異常にデータが小さい場合は処理の遅延を招く原因の一つでもあります。

そのため、パーティションの設計は適切に行う必要があります。  
良かれと思ってパーティションを綺麗に切ったつもりが実は処理の低下を招いていたということもあるのです。

In [63]:
spark.stop()